In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from datetime import datetime

np.random.seed(42)
tf.random.set_seed(42)

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPUs available: {len(gpus)}")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU found, using CPU")

IMG_SIZE = 224
BATCH_SIZE = 16
EPOCHS = 100
LEARNING_RATE = 0.001
NUM_CLASSES = 6 

TRAIN_DIR = "/mnt/d/code/hcl/NEU-DET/train/images"
VAL_DIR = "/mnt/d/code/hcl/NEU-DET/validation/images"

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=42
)

val_generator = val_datagen.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    seed=42
)

class_names = list(train_generator.class_indices.keys())
print(f"\nDataset Summary:")
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {val_generator.samples}")
print(f"Number of classes: {len(class_names)}")
print(f"Classes: {class_names}")

def build_efficientnet_model(num_classes, img_size=224):
    base_model = EfficientNetB3(
        include_top=False,
        weights='imagenet',
        input_shape=(img_size, img_size, 3)
    )
    base_model.trainable = False
    inputs = keras.Input(shape=(img_size, img_size, 3))
    x = layers.RandomFlip("horizontal_and_vertical")(inputs)
    x = layers.RandomRotation(0.1)(x)
    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = keras.Model(inputs, outputs)
    
    return model, base_model

print("\nBuilding EfficientNet-B3 model...")
model, base_model = build_efficientnet_model(NUM_CLASSES, IMG_SIZE)

model.compile(
    optimizer=keras.optimizers.AdamW(learning_rate=LEARNING_RATE, weight_decay=0.01),
    loss='categorical_crossentropy',
    metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=2, name='top_2_accuracy')]
)

print("\nModel Architecture:")
model.summary()

callbacks = [
    ModelCheckpoint(
        'best_efficientnet_model.h5',
        monitor='val_accuracy',
        mode='max',
        save_best_only=True,
        verbose=1
    ),
    
    EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    ),
    
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    ),
    
    TensorBoard(
        log_dir=f'logs/fit/{datetime.now().strftime("%Y%m%d-%H%M%S")}',
        histogram_freq=1
    )
]

print("Phase 1: Training custom head (base model frozen)")
history_phase1 = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=callbacks,
    verbose=1
)
print("Phase 2: Fine-tuning entire model")
base_model.trainable = True
model.compile(
    optimizer=keras.optimizers.AdamW(learning_rate=LEARNING_RATE/10, weight_decay=0.01),
    loss='categorical_crossentropy',
    metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=2, name='top_2_accuracy')]
)

history_phase2 = model.fit(
    train_generator,
    epochs=EPOCHS - 20,
    validation_data=val_generator,
    callbacks=callbacks,
    verbose=1,
    initial_epoch=20
)

history = {
    'accuracy': history_phase1.history['accuracy'] + history_phase2.history['accuracy'],
    'val_accuracy': history_phase1.history['val_accuracy'] + history_phase2.history['val_accuracy'],
    'loss': history_phase1.history['loss'] + history_phase2.history['loss'],
    'val_loss': history_phase1.history['val_loss'] + history_phase2.history['val_loss']
}


print("Loading best model for evaluation...")

best_model = keras.models.load_model('best_efficientnet_model.h5')

val_loss, val_accuracy, val_top2_acc = best_model.evaluate(val_generator, verbose=1)
print(f"\nBest Validation Accuracy: {val_accuracy*100:.2f}%")
print(f"st Validation Loss: {val_loss:.4f}")
print(f"p-2 Accuracy: {val_top2_acc*100:.2f}%")

print("\nGenerating predictions...")
val_generator.reset()
predictions = best_model.predict(val_generator, verbose=1)
y_pred = np.argmax(predictions, axis=1)
y_true = val_generator.classes


print("Classification Report")

print(classification_report(y_true, y_pred, target_names=class_names, digits=4))


print("Per-Class Accuracy")

cm = confusion_matrix(y_true, y_pred)
per_class_acc = cm.diagonal() / cm.sum(axis=1) * 100
for i, class_name in enumerate(class_names):
    print(f"{class_name:20s}: {per_class_acc[i]:.2f}%")

def plot_training_history(history):
    _, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    epochs_range = range(1, len(history['accuracy']) + 1)
    
    ax1.plot(epochs_range, history['accuracy'], 'b-o', label='Train Accuracy', markersize=4)
    ax1.plot(epochs_range, history['val_accuracy'], 'r-s', label='Val Accuracy', markersize=4)
    ax1.axvline(x=20, color='green', linestyle='--', label='Fine-tuning starts', alpha=0.7)
    ax1.set_xlabel('Epoch', fontsize=12)
    ax1.set_ylabel('Accuracy', fontsize=12)
    ax1.set_title('Training and Validation Accuracy', fontsize=14, fontweight='bold')
    ax1.legend(fontsize=10)
    ax1.grid(True, alpha=0.3)
    
    ax2.plot(epochs_range, history['loss'], 'b-o', label='Train Loss', markersize=4)
    ax2.plot(epochs_range, history['val_loss'], 'r-s', label='Val Loss', markersize=4)
    ax2.axvline(x=20, color='green', linestyle='--', label='Fine-tuning starts', alpha=0.7)
    ax2.set_xlabel('Epoch', fontsize=12)
    ax2.set_ylabel('Loss', fontsize=12)
    ax2.set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
    ax2.legend(fontsize=10)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("\naining history saved as 'training_history.png'")

def plot_confusion_matrix(cm, class_names):
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names,
                cbar_kws={'label': 'Count'})
    plt.title('Confusion Matrix', fontsize=16, fontweight='bold', pad=20)
    plt.ylabel('True Label', fontsize=12)
    plt.xlabel('Predicted Label', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("nfusion matrix saved as 'confusion_matrix.png'")

plot_training_history(history)
plot_confusion_matrix(cm, class_names)


print("Saving model in multiple formats...")


best_model.save('best_efficientnet_model_saved', save_format='tf')
print("del saved in SavedModel format: 'best_efficientnet_model_saved/'")

print("del saved in HDF5 format: 'best_efficientnet_model.h5'")

converter = tf.lite.TFLiteConverter.from_keras_model(best_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
with open('best_efficientnet_model.tflite', 'wb') as f:
    f.write(tflite_model)
print("del saved in TFLite format: 'best_efficientnet_model.tflite'")


print("Training Complete!")

print("st model saved in multiple formats")
print("ning history saved as 'training_history.png'")
print("nfusion matrix saved as 'confusion_matrix.png'")
print("orBoard logs saved in 'logs/' directory")
print(f"\nFinal Validation Accuracy: {val_accuracy*100:.2f}%")

def load_and_predict(image_path, model_path='best_efficientnet_model.h5'):
    model = keras.models.load_model(model_path)
    
    img = keras.preprocessing.image.load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    predictions = model.predict(img_array)
    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = np.max(predictions[0]) * 100
    
    print(f"Predicted Class: {predicted_class}")
    print(f"Confidence: {confidence:.2f}%")
    
    return predicted_class, confidence